In [1]:
from lib import *
import numpy as np

import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
from itertools import product

*** THIS NOTEBOOK CAN BE USED TO ANALYSE RAW AND DEC OP.WVFs FROM LARSOFT ***

In [2]:
raw,key_list         = load_larsoft_root("RAW","opdetraw_hist.root",     "opdigiana",np.loadtxt("template/fbk_deco.txt"))
gauss_dec,key_list   = load_larsoft_root("DEC","deconv_gauss_hist.root", "opdecoana",np.loadtxt("template/fbk_deco.txt"))
wiener_dec,key_list  = load_larsoft_root("DEC","deconv_wiener_hist.root","opdecoana",np.loadtxt("template/fbk_deco.txt"))

In [3]:
template = np.loadtxt("template/fbk_deco.txt")
# print(np.asarray(template))
np.sum(template[template > 0])

77.969230247048

In [4]:
SER = []
templates = ["fbk_digi.txt","fbk_deco.txt","fbk_1024_dapv2.txt","fbk_DAPHNE_Dec22.txt","fbk_CA_deco_sinpret.txt"]
for path in templates:
    SER.append(np.loadtxt("template/"+path))

fig = go.Figure()
for i in range(len(SER)):
    fig.add_trace(go.Scatter(name=templates[i],x=np.arange(len(SER[i])),y=SER[i]))

fig.update_layout(title_text='Template comparison',
                xaxis_title  = "Time in [ticks]",
                yaxis_title  = "Amp. in [ADC]",
)

fig.show()

*** VISUALIZE DATA ***

In [37]:
num = np.random.randint(len(raw["RECO"]["WVF"]))
fig = make_subplots(rows=1, cols=2,subplot_titles=('', ''))

fig.add_trace(go.Scatter(line=dict(color="#3366CC"),name="RAW #PE: %.2f"%(raw["RECO"]["PE"][num]),x=raw["RECO"]["WVF_X"][num],y=raw["RECO"]["WVF"][num]-raw["PEDESTAL"]),col=1,row=1)
fig.add_trace(go.Scatter(line=dict(color="#66AA00"),name="GAUSS #PE: %.2f"%(gauss_dec["RECO"]["PE"][num]),x=gauss_dec["RECO"]["WVF_X"][num],y=gauss_dec["RECO"]["WVF"][num]),col=2,row=1)
fig.add_trace(go.Scatter(line=dict(color="#FF9900"),name="WIENER #PE: %.2f"%(wiener_dec["RECO"]["PE"][num]),x=wiener_dec["RECO"]["WVF_X"][num],y=wiener_dec["RECO"]["WVF"][num]),col=2,row=1)

fig.add_vline(x=raw["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=1)
fig.add_vline(x=gauss_dec["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=2)

if len(raw["RECO"]["PETIMES"][num]) > 50:
    fig.add_trace(go.Histogram(name="TRUE #PE: %.2f"%(len(raw["RECO"]["PETIMES"][num])),x=np.asarray(raw["RECO"]["PETIMES"][num])+raw["PRETRIGGER"]*raw["SAMPLING"]*1e6),col=1,row=1)

else:
    fig.add_trace(go.Scatter(line=dict(color="#DC3912"),name="TRUE PE",mode="markers",x=np.asarray(raw["RECO"]["PETIMES"][num])+raw["PRETRIGGER"]*raw["SAMPLING"]*1e6,y=np.zeros(len(raw["RECO"]["PETIMES"][num]))),col=1,row=1)
    fig.add_trace(go.Scatter(line=dict(color="#DC3912"),name="TRUE PE",mode="markers",x=np.asarray(raw["RECO"]["PETIMES"][num])+raw["PRETRIGGER"]*raw["SAMPLING"]*1e6,y=np.zeros(len(raw["RECO"]["PETIMES"][num]))),col=2,row=1)
    
fig.update_layout(
    title = "Deco. wvf comparison",
    xaxis_title  = "Time in [&mu;s]",
    yaxis_title  = "Amp. in [ADC]",
    xaxis2_title = "Time in [&mu;s]",
    yaxis2_title = "Amp. in [a.u.]",
)
fig.show()

*** CONFIGURE FILTER FOR ANALYSIS ***

In [30]:
PE_true = np.append(raw["TRUE"]["PE"],np.append(raw["TRUE"]["PE"],raw["TRUE"]["PE"]))
amp = np.append(raw["RECO"]["AMP"],np.append(gauss_dec["RECO"]["AMP"],wiener_dec["RECO"]["AMP"]))
PE_reco = np.append(raw["RECO"]["PE"],np.append(gauss_dec["RECO"]["PE"],wiener_dec["RECO"]["PE"]))
label_amp = np.append(["RAW"]*len(raw["TRUE"]["PE"]),(np.append(["GAUSS"]*len(raw["TRUE"]["PE"]),["WIENER"]*len(raw["TRUE"]["PE"]))))
reco_amp = pd.DataFrame({"CHARGE":PE_reco, "AMP":amp, "FILTER":label_amp, "TRUE_PE":PE_true})

fig = px.scatter(data_frame=reco_amp[reco_amp['TRUE_PE'] > 0], y='TRUE_PE', x="CHARGE",color='FILTER', 
          marginal_y='histogram', marginal_x='histogram',trendline="ols")
fig.show()

results = px.get_trendline_results(fig)
print(results)
results.query("FILTER == 'RAW'").px_fit_results.iloc[0].summary()

   FILTER                                     px_fit_results
0     RAW  <statsmodels.regression.linear_model.Regressio...
1   GAUSS  <statsmodels.regression.linear_model.Regressio...
2  WIENER  <statsmodels.regression.linear_model.Regressio...


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.438e+05
Date:                Thu, 09 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:18:12   Log-Likelihood:                -2030.2
No. Observations:                 481   AIC:                             4064.
Df Residuals:                     479   BIC:                             4073.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.0793      1.015     -9.934      0.000     -12.073      -8.086
x1             2.6365      0.007    379.263      0.000       2.623       2.650
==============================================================================
Omnibus:                       47.913   Durbin-Watson:                   1.752
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.371
Skew:                          -0.142   Prob(JB):                     1.06e-57
Kurtosis:                       6.607   Cond. No.                         197.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
PE_error_raw = (np.asarray(raw["RECO"]["PE"])-np.asarray(raw["TRUE"]["PE"]))/np.asarray(raw["TRUE"]["PE"])
PE_error_gauss = (np.asarray(gauss_dec["RECO"]["PE"])-np.asarray(raw["TRUE"]["PE"]))/np.asarray(raw["TRUE"]["PE"])
PE_error_wiener = (np.asarray(wiener_dec["RECO"]["PE"])-np.asarray(raw["TRUE"]["PE"]))/np.asarray(raw["TRUE"]["PE"])

PE_true = np.append(raw["TRUE"]["PE"],np.append(raw["TRUE"]["PE"],raw["TRUE"]["PE"]))
PE_error = 100*np.append(PE_error_raw,np.append(PE_error_gauss,PE_error_wiener))
label_error = np.append(["RAW"]*len(PE_error_raw),np.append(["GAUSS"]*len(PE_error_gauss),["WIENER"]*len(PE_error_gauss)))
reco_PE = pd.DataFrame({"'%'ERROR":PE_error, "FILTER":label_error, "TRUE_PE":PE_true})

px.scatter(data_frame=reco_PE[reco_PE['TRUE_PE'] > 10], y='TRUE_PE', x="'%'ERROR",color='FILTER', marginal_x='histogram')

In [11]:
PE_filter = np.where(np.asarray(raw["TRUE"]["PE"]) > 0)

fig = go.Figure()

fig.add_trace(go.Histogram(name="RAW"   ,x=np.asarray(raw["RECO"]["T0"])[PE_filter]       - raw["PRETRIGGER"]*raw["SAMPLING"]*1e6,xbins=dict(start=-0.16,end=0.32,size=0.016)))
fig.add_trace(go.Histogram(name="GAUSS" ,x=np.asarray(gauss_dec["RECO"]["T0"])[PE_filter] - raw["PRETRIGGER"]*raw["SAMPLING"]*1e6,xbins=dict(start=-0.16,end=0.32,size=0.016)))
fig.add_trace(go.Histogram(name="WIENER",x=np.asarray(wiener_dec["RECO"]["T0"])[PE_filter]- raw["PRETRIGGER"]*raw["SAMPLING"]*1e6,xbins=dict(start=-0.16,end=0.32,size=0.016)))

fig.update_layout(title_text='T0 recovery',
                xaxis_title  = "Time in [&mu;s]",
                yaxis_title  = "Counts"
)

fig.update_traces(opacity=0.75)
fig.show()